# Accu.Tuning Tutorial 1 - Binary Classification

**Accu.Tuning**을 이용하여 해결할 수 있는 여러 문제 유형 중 분류, 그 중에서도 분류 유형이 두 가지 있는 이진 분류(Binary Classification)를 시행합니다. **Accu.Tuning** 상에서 고객 데이터를 이용하여 각 고객의 이탈 여부(Yes or No)를 예측하는 간단한 예제를 소개합니다.

## 1. 데이터 준비

Kaggle에서 구할 수 있는 Churn dataset을 활용하여 튜토리얼을 진행합니다.

In [1]:
import os
import numpy as np
import pandas as pd

사용할 데이터의 형태는 다음과 같습니다. 목표 변수는 Yes, No 두 가지로 구분되는 'Churn' 컬럼입니다.

In [2]:
df = pd.read_csv('resources/classification_data.csv')
df.head(10)

,CustomerID,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,...,ReferralsMadeBySubscriber,IncomeGroup,OwnsMotorcycle,AdjustmentsToCreditRating,MadeCallToRetentionTeam,CreditRating,PrizmCode,Occupation,MaritalStatus,Churn
0,3000002,24.00,219.0,22.0,0.25,0.0,0.0,-157.0,-19.0,0.7,...,0,4,No,0,Yes,1-Highest,Suburban,Professional,No,Yes
1,3000010,16.99,10.0,17.0,0.00,0.0,0.0,-4.0,0.0,0.3,...,0,5,No,0,No,4-Medium,Suburban,Professional,Yes,Yes
2,3000014,38.00,8.0,38.0,0.00,0.0,0.0,-2.0,0.0,0.0,...,0,6,No,0,No,3-Good,Town,Crafts,Yes,No
3,3000022,82.28,1312.0,75.0,1.24,0.0,0.0,157.0,8.1,52.0,...,0,6,No,0,No,4-Medium,Other,Other,No,No
4,3000026,17.14,0.0,17.0,0.00,0.0,0.0,0.0,-0.2,0.0,...,0,9,No,1,No,1-Highest,Other,Professional,Yes,Yes
5,3000030,38.05,682.0,52.0,0.25,0.0,0.0,148.0,-3.1,9.0,...,0,1,No,1,No,3-Good,Other,Other,Yes,No
6,3000038,31.66,26.0,30.0,0.25,0.0,0.0,60.0,4.0,0.0,...,0,9,No,1,No,1-Highest,Other,Self,Yes,No
7,3000042,62.13,98.0,66.0,2.48,0.0,0.0,24.0,6.8,0.0,...,0,6,No,0,No,1-Highest,Other,Professional,No,No
8,3000046,35.30,24.0,35.0,0.00,0.0,0.0,20.0,-0.3,0.0,...,0,9,No,0,No,1-Highest,Other,Other,Yes,No
9,3000050,81.00,1056.0,75.0,0.00,0.0,0.0,43.0,2.4,0.0,...,0,5,No,1,No,3-Good,Other,Professional,No,No


In [3]:
df.shape

(10000, 57)

## 2. 데이터 전처리

데이터가 준비되었다면 모델링을 시작하기에 앞서 전처리를 수행할 수 있습니다. 전처리 과정은 문자열로 구성된 데이터를 ML이 처리할 수 있는 숫자 형태로 변환을 하거나, 결측값을 처리하는 등의 과정을 통하여 모델 학습에 적합한 형태로 데이터를 변환하는 과정입니다. **Accu.Tuning**의 전처리 기능 상에서는 사용자가 직접 화면 상에서 전처리 과정을 설정할 수도 있고, 자동 추천 기능을 통하여 필요한 변환이 어떤 것이 있을지 제안을 받을 수도 있습니다. 해당 예제에서는 추천 기능을 통하여 전처리 파이프라인을 자동으로 구성합니다.

추천 기능을 통해 자동 구성된 데이터 전처리 파이프라인은 다음과 같습니다.

![pipeline](img/classification_pipeline.png)


- 필요한 경우 변수 데이터 타입을 수정하여 모델 학습에 보다 적합하도록 수정할 수 있습니다. 해당 데이터셋의 경우 UniqueSubs, ActiveSubs 등 Integer로 구성되어 있는 변수들이지만 변수의 고유값 개수에 따라 범주형 데이터로 이해하여 Object로 변환하고 있습니다. (convert_dtypes)

- 결측값이 존재할 경우 이를 중간값, 평균, 혹은 다른 값을 채워넣어 데이터의 완결성을 보완할 수 있습니다. (impute_nulls)

- 연속형 숫자 변수는 Box-Cox 변환을 하거나 로그 변환을 하는 등 정규화하여 필요한 경우에 선형 모델에서 더 나은 성능을 보이도록 변환할 수 있고, 범주형 변수의 경우 범주 개수에 따라 정수로 변환(IntegerEncoding)되거나 각각의 범주 컬럼으로 변환(OneHotEncoding)할 수 있습니다.

- 자동 구성된 데이터 전처리 과정 외에도 사용자의 필요에 따라 여러 전처리 과정을 추가할 수 있습니다.

#### 사용자가 전처리 설정에 참고할 수 있도록 Accu.Tuning 상에서는 아래와 같이 상관행렬, 변수분석 등 EDA (탐색적 데이터 분석)를 위한 플롯을 제공하고 있습니다.

![corr_plot](img/classification_corr_plot.png)

![var_anal](img/classification_var_anal.png)

## 3. 모델링

**Accu.Tuning** 상에서는 모델링에 앞서 사용자 필요에 따라 모델 구성 설정을 변경할 수 있습니다.

![model_configs](img/classification_model_configs.png)

**Accu.Tuning** 상에서 **RUN AUTOML** 버튼을 클릭하여 바로 모델링을 시작할 수 있습니다. 그 결과로 총 35 개의 모델이 구성되었고, 아래 Leaderboard에서 테스트셋 Accuracy 기준 최고 모델은 Random Forest Classifier 임을 확인할 수 있습니다.

![results](img/classification_results.png)

**Accu.Tuning**의 기능들은 기본적으로 웹페이지 상 UI를 통해 제공되지만, 아래와 같이 Jupyter Notebook 등에서 사용자가 직접 **Accu.Tuning** API를 활용하는 코드를 작성해 사용할 수도 있습니다.

In [4]:
r = Runtime.objects.get(pk=5)

In [5]:
pd.DataFrame([
    dict(
        estimator_name=mdl.estimator_name,
        train_score=mdl.train_score,
        test_score=mdl.score,
    )
    for mdl in r.ml_models.order_by('-score', '-train_score')
])

,estimator_name,train_score,test_score
0,random_forest,0.911143,0.714333
1,decision_tree,0.711143,0.714333
2,decision_tree,0.711143,0.714333
3,decision_tree,0.711143,0.714333
4,decision_tree,0.711143,0.714333
5,decision_tree,0.711143,0.714333
6,decision_tree,0.711143,0.714333
7,decision_tree,0.711143,0.714333
8,decision_tree,0.711143,0.714333
9,random_forest,0.966143,0.714000


본 Experiment 상에서 모델 선정 기준으로 활용한 Accuracy 외에도 모델을 통해 구할 수 있는 다른 평가 지표들 또한 아래와 같이 확인할 수 있습니다.

In [6]:
best_model = r.ml_models.order_by('-score', '-train_score').first()

In [7]:
import json
pd.DataFrame(
    json.loads(best_model.all_metrics_json).items(),
    columns = ['Metrics', '[Train Score, Test Score]']
)

,Metrics,"[Train Score, Test Score]"
0,accuracy,"[0.9111, 0.7143]"
1,roc_auc,"[0.8522, 0.5396]"
2,balanced_accuracy,"[0.8522, 0.5396]"
3,average_precision,"[0.7858, 0.3204]"
4,neg_log_loss,"[3.069, 9.8666]"
5,precision,"[0.9879, 0.535]"
6,precision_macro,"[0.9392, 0.6311]"
7,precision_micro,"[0.9111, 0.7143]"
8,precision_samples,"[0.0, 0.0]"
9,precision_weighted,"[0.9193, 0.6714]"


평가 지표들 외에도 **Accu.Tuning**은 아래와 같이 모델을 평가할 수 있는 다양한 플롯을 제공하여 모델 결과를 다양한 시각에서 파악할 수 있도록 합니다. 분류 문제에서 기본적으로 활용되는 Confusion Matrix나 ROC Curve 등의 플롯과 모델에서 주요하게 활용된 변수를 파악할 수 있는 피쳐 중요도 등을 제시함으로써 사용자가 효과적으로 모델을 평가할 수 있도록 지원합니다.

![plot1](img/classification_result_plot.png)
![plot2](img/classification_feature_importance.png)

## 4. 모델 활용 및 결과 예측

학습이 다 이루어졌다면, 완료된 모델을 Binary 형태로 불러와 **Accu.Tuning** 기능 밖에서도 활용할 수 있습니다. Leaderboard 상에서 **배포** 버튼을 클릭하면 구성된 모델을 pipeline.pkl 파일로 다운로드할 수 있습니다.

외부 활용을 지원하기 위하여 아래와 같이 Accu.Tuning의 도구 모음인 autoinsight_helpers 패키지를 설치해 이용할 수 있습니다.

In [8]:
!pip install \
--trusted-host github.com \
--trusted-host codeload.github.com \
https://github.com/skcc-asset-platform/autoinsight_helpers/archive/v1.0.17.zip

  Using cached https://github.com/skcc-asset-platform/autoinsight_helpers/archive/v1.0.17.zip


In [9]:
import pickle
pipeline = pickle.load(open('resources/classification_pipeline.pkl', 'rb'))
print(pipeline) # CHECK PIPELINE STRUCTURE

Pipeline(steps=[('pp-0',
                 Pipeline(steps=[('convert_dtypes',
                                  AutoinsightDtypeConvert(datatype_pair_match=[('UniqueSubs',
                                                                                'object'),
                                                                               ('ActiveSubs',
                                                                                'object'),
                                                                               ('HandsetModels',
                                                                                'object'),
                                                                               ('RetentionCalls',
                                                                                'object'),
                                                                               ('RetentionOffersAccepted',
                                                                     

불러온 모델을 통하여 각 행별로, 혹은 데이터프레임을 통한 예측을 아래와 같이 실시할 수 있습니다.

In [10]:
df.drop(['Churn'], axis=1, inplace=True) # 기존 데이터프레임에서 새로 예측하고자 하는 열을 삭제합니다.

In [11]:
pipeline.predict(df)

CRITICAL:root:No such column name in the dataset - Null Imputer


array([1, 1, 0, ..., 0, 1, 1])